# 🔐 Feistel Cipher + Basit DES Mantığı ile Şifreleme
Bu notebook, Feistel şifre yapısını ve basitleştirilmiş bir DES algoritmasını Python ile sıfırdan uygular. Tüm işlemler adım adım açıklamalıdır.

In [ ]:
import random

def xor(a: int, b: int, bits=32) -> int:
    return a ^ b

# S-box (tek tablo)
S_BOX = [
    [0xE, 0x4, 0xD, 0x1,
     0x2, 0xF, 0xB, 0x8,
     0x3, 0xA, 0x6, 0xC,
     0x5, 0x9, 0x0, 0x7]
]

def apply_sbox(input32: int) -> int:
    output = 0
    for i in range(8):
        shift = 28 - 4*i
        nibble = (input32 >> shift) & 0xF
        substituted = S_BOX[0][nibble]
        output = (output << 4) | substituted
    return output

def permute(input32: int) -> int:
    return input32

def f_function(right: int, subkey: int) -> int:
    return permute(apply_sbox(xor(right, subkey)))

def split_block(block: bytes) -> tuple:
    full = int.from_bytes(block, 'big')
    left = (full >> 32) & 0xFFFFFFFF
    right = full & 0xFFFFFFFF
    return left, right

def join_block(left: int, right: int) -> bytes:
    full = (left << 32) | right
    return full.to_bytes(8, 'big')

def generate_round_keys(main_key: int, rounds: int = 16) -> list:
    return [main_key for _ in range(rounds)]

def feistel_encrypt(block: bytes, key: int, rounds: int = 16) -> bytes:
    left, right = split_block(block)
    round_keys = generate_round_keys(key, rounds)
    for i in range(rounds):
        new_left = right
        right = xor(left, f_function(right, round_keys[i]))
        left = new_left
    return join_block(right, left)

def feistel_decrypt(block: bytes, key: int, rounds: int = 16) -> bytes:
    right, left = split_block(block)
    round_keys = generate_round_keys(key, rounds)
    for i in reversed(range(rounds)):
        new_right = left
        left = xor(right, f_function(left, round_keys[i]))
        right = new_right
    return join_block(left, right)

In [ ]:
# Örnek demo
plaintext = b"APO123!!"
key = random.getrandbits(32)

print("Orijinal     :", plaintext)
encrypted = feistel_encrypt(plaintext, key)
print("Şifreli (hex):", encrypted.hex())
decrypted = feistel_decrypt(encrypted, key)
print("Çözülmüş     :", decrypted)

assert decrypted == plaintext